In [10]:
import numpy
import tensorflow as tf
from numpy import array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding

In [11]:
# 감정 분석
# 텍스트 리뷰 자료 지정
docs=['너무 재밋네요','최고예요','참 잘 만든 영화예요','추천하고 싶은 영화입니다.',
      '한번 더 보고싶네요','글쎄요','별로예요','생각보다 지루하네요','연기가 어색해요',
      '재미없어요']

In [12]:
# 감정사전 : 긍정 리뷰는 1, 부정 리뷰는 0으로 클래스 지정
classes = array([1,1,1,1,1,0,0,0,0,0])

In [13]:
# 토큰화(전처리)
token = Tokenizer()
token.fit_on_texts(docs)

# 단어의 빈도수 개산한 결과를 각 옵션에 맞추어 출력
print(token.word_index) # 토큰화 된 결과를 출력해 확인
# Tokenizer()의 word_counts 함수는 순서를 기억하는 OrderedDict 클래스 사용함

{'너무': 1, '재밋네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로예요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}


In [15]:
# 토큰에 지정된 인덱스로 새로운 배열 생성
x = token.texts_to_sequences(docs)
print(x)
# 입력된 리뷰 데이터의 토큰 수가 각각 다름
# 딥러닝 모델에 입력하기 위해선 학습 데이터의 길이가 동일해야함

[[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13], [14], [15], [16, 17], [18, 19], [20]]


In [18]:
# 패딩, 서로 다른 길이의 데이터를 4로 맞춤
padded_x = pad_sequences(x,4)
print(padded_x)

[[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [ 0 11 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0  0 20]]


In [20]:
# 임베딩에 입력될 단어 수 지정
## 임베딩 함수에 필요한 세가지 파라미터는 '입력,출력,단어 수'


# '입력' 설정
# 전체 단어의 맨 앞에 0이 먼저 나와야 하므로 총 단어 수+1 명심
word_size = len(token.word_index)+1

# '출력' 설정
#Embedding(word_size, 8, input_length=4)

In [24]:
# 단어 임베딩을 포함하여 딥러닝 모델을 만들고 결과 출력
model = Sequential()
model.add(Embedding(word_size, 8, input_length=4)) 
# input_length=4 패딩된 입력값(index) 크기가 4이기 때문
model.add(Flatten()) # 차원축소 (2차원>>1차원)
model.add(Dense(1, activation='sigmoid')) # 출력층, 이진 분류(sigmoid)

# 모델 실행 환경 설정
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

# 모델 학습 (fit)

model.fit(padded_x,classes, epochs=20)
print("\n Accuracy: %.4f"% (model.evaluate(padded_x, classes)[1]))

Epoch 1/20
1/1 [==============================] - 1s 526ms/step - loss: 0.6904 - accuracy: 0.4000
Epoch 2/20
1/1 [==============================] - 0s 12ms/step - loss: 0.6884 - accuracy: 0.5000
Epoch 3/20
1/1 [==============================] - 0s 10ms/step - loss: 0.6865 - accuracy: 0.6000
Epoch 4/20
1/1 [==============================] - 0s 11ms/step - loss: 0.6845 - accuracy: 0.6000
Epoch 5/20
1/1 [==============================] - 0s 13ms/step - loss: 0.6825 - accuracy: 0.6000
Epoch 6/20
1/1 [==============================] - 0s 14ms/step - loss: 0.6806 - accuracy: 0.6000
Epoch 7/20
1/1 [==============================] - 0s 10ms/step - loss: 0.6786 - accuracy: 0.6000
Epoch 8/20
1/1 [==============================] - 0s 11ms/step - loss: 0.6767 - accuracy: 0.6000
Epoch 9/20
1/1 [==============================] - 0s 12ms/step - loss: 0.6747 - accuracy: 0.6000
Epoch 10/20
1/1 [==============================] - 0s 12ms/step - loss: 0.6727 - accuracy: 0.7000
Epoch 11/20
1/1 [===========

In [23]:
# 결과해석
# 학습결과 Accuracy: 0.9000
# 90%의 정확도를 보여줌